# Tool Use

Goal:
- Teach claude to send reminders
- Implement 3-4 custom tools

Some challenges:
- Claude might know the date, but not the time.
- Time-based addition is hard for Claude

Tools to build:
- Get the current date time
- Add duration to date time
- Set a reminder

#### Imports and constants

In [2]:
import json
from datetime import datetime, timedelta
from IPython.display import Markdown
from anthropic import Anthropic
from dotenv import load_dotenv

# Load environment variables
'''
.env file must contain ANTHROPIC_API_KEY
'''
load_dotenv()

from lib.ClaudeResponsesClass import ClaudeResponses
from anthropic.types import ToolParam, Message


In [3]:
client = Anthropic()
model = "claude-3-7-sonnet-latest"
claude = ClaudeResponses(save_history=False, model=model)

##### Some examples of Claude and time

In [4]:
claude.chat("hello")

"Hello! How are you today? I'm here to help with any questions or tasks you might have. Just let me know what you're looking for, and I'll do my best to assist you."

In [5]:
claude.chat("what time is it?") 

"I don't have the ability to check the current time. I don't have access to real-time data or your location information. To find out the current time, you could check your device's clock, a watch, or use a time service online."

In [6]:
claude.chat("what day is it?")

'Today is Friday, May 10, 2024.'

## Creating a Tool for Claude

##### 1. Program the function

**Tool function:** Plain Python function that will be executed when Claude decides it needs some additioal info to help the user.
- Use well-named, descriptive arguments
- Validate inputs, raising an error if they fail validation
- Return meaningful errors - Claude will try to call to use the function a second time, it should understand how to fix it.

In [5]:
def get_current_datetime(date_format: str = "%Y-%m-%d %H:%M:%S"):
    from datetime import datetime
    # Validate
    if not date_format:
        raise ValueError("date_format cannot be empty")
    return datetime.now().strftime(date_format)

In [5]:
get_current_datetime("%H:%M")

'12:51'

##### 2. Write a JSON Schema

The JSON schema helps Claude understand what arguments the function requires
*Commonly used for data validation.

- Explain what the tool does, when to use it and what it returns.
- Aim for 3-4 sentences.
- Provide super detailed descriptions

*Claude can generate a JSON schema based on its documentation for best practices.*

"Write a valid JSON schema spec for the purposes of tool calling for this function. Follow the best practices listed in the attached documentation."

In [6]:
# ToolParam avoids TypeError
get_current_datetime_schema = ToolParam({
  "name": "get_current_datetime",
  "description": "Retrieves the current date and time formatted according to a specified format string. This tool should be used when the user asks for the current time, current date, or current datetime information. The tool uses Python's strftime format codes to control the output format. Common format codes include %Y (4-digit year), %m (month), %d (day), %H (24-hour), %M (minutes), %S (seconds). The default format returns datetime as 'YYYY-MM-DD HH:MM:SS'. The tool will raise an error if an empty format string is provided. Use this tool whenever you need to provide real-time temporal information to the user.",
  "input_schema": {
    "type": "object",
    "properties": {
      "date_format": {
        "type": "string",
        "description": "The strftime format string that determines how the datetime will be formatted in the output. Must be a non-empty string using Python strftime format codes. Examples: '%Y-%m-%d' for '2023-12-25', '%I:%M %p' for '3:45 PM', '%A, %B %d, %Y' for 'Monday, December 25, 2023'. Defaults to '%Y-%m-%d %H:%M:%S' if not specified.",
        "default": "%Y-%m-%d %H:%M:%S"
      }
    },
    "required": []
  }
})

#### 3. Call Claude with JSON Schema

In [7]:
messages = []
messages.append(
    {
        "role": "user",
        "content": "Hello Claude! How are you? can you tell me the time? (just the hour and minutes)"
    }
)

response = client.messages.create(
    model=model,
    max_tokens=1000,
    messages=messages,
    tools = [get_current_datetime_schema]
)

response

Message(id='msg_019jf7zKLeuYggYYtdz6Nkg4', content=[TextBlock(citations=None, text="Hello! I'm doing well, thank you for asking. I'd be happy to tell you the current time with just the hour and minutes.", type='text'), ToolUseBlock(id='toolu_01SjmjMC1Mo79gvnyQmgwjgz', input={'date_format': '%H:%M'}, name='get_current_datetime', type='tool_use')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=662, output_tokens=90, server_tool_use=None, service_tier='standard'))

ToolUseBlock shows Claude wants to use a tool.

In [8]:
# In this case, no TextBlock is created, as Claude considers the output of the tool is enough.
response.content

[TextBlock(citations=None, text="Hello! I'm doing well, thank you for asking. I'd be happy to tell you the current time with just the hour and minutes.", type='text'),
 ToolUseBlock(id='toolu_01SjmjMC1Mo79gvnyQmgwjgz', input={'date_format': '%H:%M'}, name='get_current_datetime', type='tool_use')]

In [43]:
messages.append(
    {
        "role": "assistant",
        "content": response.content
    }
)

messages

[{'role': 'user',
  'content': 'Hello Claude! How are you? can you tell me the time? (just the hour and minutes)'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text='Certainly! Here is the current time:', type='text'),
   ToolUseBlock(id='toolu_01Ggx2DSzfycZDLEZCESy4BE', input={'date_format': '%I:%M %p'}, name='get_current_datetime', type='tool_use')]}]

In [45]:
# We can see the type of block Claude returns
[getattr(c, "type") for c in response.content]

['text', 'tool_use']

#### 4. Run Tool Function Claude Requested

**Tool Use Block**: 

{

    "id": Tool use ID, 
    "type": "tool_use", 
    "input": Dictionary with the arguments of the tool, 
    "name": Name of the tool to run. 
}

**Tool Result Block**: 
- Placed in the content list of user message.
- Communicates the results of running a tool function back to Claude
- **tool_use_id** - Must match the id of the ToolUse block that this ToolResult corresponds to
- **"content"** - Output from running your tool, serialized as a string
- **"is_error"** - True if an error occured

*Ex:*
{
    
    "tool_use_id": Tool use ID, 
    "type": "tool_result", 
    "content": Output from the tool call, enconded as string, 
    "is_error": True if an error occured. 
}

In [50]:
response.content

[TextBlock(citations=None, text='Certainly! Here is the current time:', type='text'),
 ToolUseBlock(id='toolu_01Ggx2DSzfycZDLEZCESy4BE', input={'date_format': '%I:%M %p'}, name='get_current_datetime', type='tool_use')]

In [ ]:
messages.append(
    {
        "role": "user",
        "content":[
            {"type": "tool_result",
             "tool_use_id": content.id,
             "content": json.dumps(str(eval(f"{content.name}(**{content.input})"))),
             # TODO - Condition if it's error
             "is_error": False
            }
            for content in response.content if content.type == "tool_use"
        ]
    }
)

In [88]:
messages

[{'role': 'user',
  'content': 'Hello Claude! How are you? can you tell me the time? (just the hour and minutes)'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text='Certainly! Here is the current time:', type='text'),
   ToolUseBlock(id='toolu_01Ggx2DSzfycZDLEZCESy4BE', input={'date_format': '%I:%M %p'}, name='get_current_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01Ggx2DSzfycZDLEZCESy4BE',
    'content': '01:07 PM',
    'is_error': False}]}]

In [94]:
final_resp = client.messages.create(
    model=model,
    max_tokens=1000,
    messages=messages,
    tools=[get_current_datetime_schema]
)

In [99]:
text_parts = [b.text for b in final_resp.content if getattr(b, "type", "text") == "text"]
response_text = "".join(text_parts) if text_parts else ""

response_text

"Hello! I'm doing well, thank you for asking. The current time is 1:07 PM."

## Multi-Turn Conversation with tools

**Conversation with tools**
- Provide an initial list of messages
- Feed messages into Claude
- If Claude isn't asking for a tool use, then we must have a final answer 
- If Claude wants to use a tool, then we run the tool, put the results into a user message, and run Claude again

*Tool use improvements*:
1. Ensure "add_user_message" and "add_assistant_message" can handle multiple message blocks
2. Allow "chat" to receive a list of tools and return the full generated message
3. Add a "text_from_message" function to extract all text from text blocks from a message
4. Add support for multiple tool calls in a conversation

### 1-3: Helper functions

In [53]:
# 1. helper functions can handle multiple message blocks

# Add content from a response, else add the text provided
def add_user_message(messages, message):
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message
    }
    messages.append(user_message)

def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message
    }
    messages.append(assistant_message)

In [54]:
# 2. chat can receive a list of tools and return full Message
def chat(messages: list,
        model: str = model,
        max_tokens: int = 1000,
        temperature: float = 1.0,
        stop_sequences = [],
        system: str | None = None,
        tools: list | None = None):
    params = {
        "model": model,
        "max_tokens": max_tokens,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences
    }
    if system:
        params["system"] = system
    if tools:
        params["tools"] = tools
    
    response = client.messages.create(**params)
    return response

In [55]:
# 3. helper function that can extract text from all text message blocks
def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

### 4: Conversation with tools

We can identify tool call from *stop_reason*:

| Stop Reason | Description |
|--|--|
| *tool_use* | Request to call a tool |
| *end_turn* | Finished generating assistant message |
| *max_tokens* | Token output limit reached - can't generate any more output |
| *stop_sequence* | Encountered one of the provided stop sequences |

#### 4.1 Support multiple tool calls

In [56]:
# 4.0 run tools
def run_tools(message):
    tool_requests = [
        block for block in message.content if block.type == "tool_use"
    ]
    tool_result_blocks = []
    for tool_request in tool_requests:
        try:
            tool_output = eval(f"{tool_request.name}(**{tool_request.input})")
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": json.dumps(tool_output),
                "is_error": False
            }
        except Exception as e:
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": f"Error: {e}",
                "is_error": True
            }
        
        tool_result_blocks.append(tool_result_block)

    return tool_result_blocks
                 

In [57]:
# 4. function that can handle multiple tool calls
def run_conversation(messages, tools: list | None = None):
    while True:
        response = chat(messages, tools=tools)

        add_assistant_message(messages, response)
        display(Markdown(text_from_message(response)))

        if response.stop_reason != "tool_use":
            break

        tool_results = run_tools(response)
        add_user_message(messages, tool_results)
    
    return messages

In [58]:
messages = []
add_user_message(messages, "What is the current time in HH:MM? What is the time in SS format?")
tools = [get_current_datetime_schema]
resp = run_conversation(messages, tools)

I can provide you with the current time in the formats you requested. Let me get that information for you.

For the time in seconds format:

The current time is 13:39, and the current seconds value is 42.

In [34]:
for message in resp:
    print(message["role"]+": ", message["content"], "\n")

user:  What is the current time in HH:MM? What is the time in SS format? 

assistant:  [TextBlock(citations=None, text="I'll get the current time in both formats you requested.", type='text'), ToolUseBlock(id='toolu_01DqUAWJd7iQngcxBpCAWKM2', input={'date_format': '%H:%M'}, name='get_current_datetime', type='tool_use'), ToolUseBlock(id='toolu_01U9Bxod3CE49NGgUzuEzr8f', input={'date_format': '%S'}, name='get_current_datetime', type='tool_use')] 

user:  [{'type': 'tool_result', 'tool_use_id': 'toolu_01DqUAWJd7iQngcxBpCAWKM2', 'content': '"14:21"', 'is_error': False}, {'type': 'tool_result', 'tool_use_id': 'toolu_01U9Bxod3CE49NGgUzuEzr8f', 'content': '"07"', 'is_error': False}] 

assistant:  [TextBlock(citations=None, text='The current time is:\n- In HH:MM format: **14:21**\n- In SS format (seconds): **07**', type='text')] 



#### 4.2 Create and call new tools.

In [14]:
# New tools
def add_duration_to_datetime(datetime_str, duration=0, unit="days", input_format="%Y-%m-%d"):
    date = datetime.strptime(datetime_str, input_format)

    if unit == "seconds":
        new_date = date + timedelta(seconds=duration)
    elif unit == "minutes":
        new_date = date + timedelta(minutes=duration)
    elif unit == "hours":
        new_date = date + timedelta(hours=duration)
    elif unit == "days":
        new_date = date + timedelta(days=duration)
    elif unit == "weeks":
        new_date = date + timedelta(weeks=duration)
    elif unit == "months":
        month = date.month + duration
        year = date.year + month // 12
        month = month % 12
        if month == 0:
            month = 12
            year -= 1
        day = min(
            date.day,
            [
                31,
                29
                if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)
                else 28,
                31,
                30,
                31,
                30,
                31,
                31,
                30,
                31,
                30,
                31,
            ][month - 1],
        )
        new_date = date.replace(year=year, month=month, day=day)
    elif unit == "years":
        new_date = date.replace(year=date.year + duration)
    else:
        raise ValueError(f"Unsupported time unit: {unit}")

    return new_date.strftime("%A, %B %d, %Y %I:%M:%S %p")

def set_reminder(content, timestamp):
    print(
        f"----\nSetting the following reminder for {timestamp}:\n{content}\n----"
    )

add_duration_to_datetime_schema = {
    "name": "add_duration_to_datetime",
    "description": "Adds a specified duration to a datetime string and returns the resulting datetime in a detailed format. This tool converts an input datetime string to a Python datetime object, adds the specified duration in the requested unit, and returns a formatted string of the resulting datetime. It handles various time units including seconds, minutes, hours, days, weeks, months, and years, with special handling for month and year calculations to account for varying month lengths and leap years. The output is always returned in a detailed format that includes the day of the week, month name, day, year, and time with AM/PM indicator (e.g., 'Thursday, April 03, 2025 10:30:00 AM').",
    "input_schema": {
        "type": "object",
        "properties": {
            "datetime_str": {
                "type": "string",
                "description": "The input datetime string to which the duration will be added. This should be formatted according to the input_format parameter.",
            },
            "duration": {
                "type": "number",
                "description": "The amount of time to add to the datetime. Can be positive (for future dates) or negative (for past dates). Defaults to 0.",
            },
            "unit": {
                "type": "string",
                "description": "The unit of time for the duration. Must be one of: 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', or 'years'. Defaults to 'days'.",
            },
            "input_format": {
                "type": "string",
                "description": "The format string for parsing the input datetime_str, using Python's strptime format codes. For example, '%Y-%m-%d' for ISO format dates like '2025-04-03'. Defaults to '%Y-%m-%d'.",
            },
        },
        "required": ["datetime_str"],
    },
}

set_reminder_schema = {
    "name": "set_reminder",
    "description": "Creates a timed reminder that will notify the user at the specified time with the provided content. This tool schedules a notification to be delivered to the user at the exact timestamp provided. It should be used when a user wants to be reminded about something specific at a future point in time. The reminder system will store the content and timestamp, then trigger a notification through the user's preferred notification channels (mobile alerts, email, etc.) when the specified time arrives. Reminders are persisted even if the application is closed or the device is restarted. Users can rely on this function for important time-sensitive notifications such as meetings, tasks, medication schedules, or any other time-bound activities.",
    "input_schema": {
        "type": "object",
        "properties": {
            "content": {
                "type": "string",
                "description": "The message text that will be displayed in the reminder notification. This should contain the specific information the user wants to be reminded about, such as 'Take medication', 'Join video call with team', or 'Pay utility bills'.",
            },
            "timestamp": {
                "type": "string",
                "description": "The exact date and time when the reminder should be triggered, formatted as an ISO 8601 timestamp (YYYY-MM-DDTHH:MM:SS) or a Unix timestamp. The system handles all timezone processing internally, ensuring reminders are triggered at the correct time regardless of where the user is located. Users can simply specify the desired time without worrying about timezone configurations.",
            },
        },
        "required": ["content", "timestamp"],
    },
}


In [16]:
messages = []
tools = [get_current_datetime_schema, add_duration_to_datetime_schema, set_reminder_schema]

add_user_message(messages, "Set a reminder for my doctor's appointment. It's 177 days after Jan1st, 2050.")

resp = run_conversation(messages, tools)

I'll help you set a reminder for your doctor's appointment. First, I need to calculate the date that is 177 days after January 1st, 2050, and then I can set the reminder for you.

Now that I have the date for your appointment, I'll set a reminder for you.

----
Setting the following reminder for 2050-06-27T00:00:00:
Doctor's appointment
----


I've set a reminder for your doctor's appointment on Monday, June 27, 2050. You'll receive a notification at midnight (12:00 AM) on that day. Is there any specific time for your appointment that you'd like to add to the reminder?

In [ ]:
def print_messages(messages) -> None:
    for message in messages:
        print(message["role"]+": ", message["content"], "\n")

print_messages(resp)

The assistant will make two different calls to its available tools. We'll give it a tool for batch calling.

#### 4.3 Batch Tool

The solution is to implement a batch tool that encourages Claude to request multiple tool calls at once.

A batch tool is an additional tool that accepts a list of calls to other tools. Instead of Claude calling multiple tools directly, it calls the batch tool and provides arguments describing which tools it wants to run.

In [18]:
def batch_tool(invocations: list = []):
    batch_outputs = []
    for invocation in invocations:
        name = invocation["name"]
        args = invocation["arguments"]
        tool_output = eval(f"{name}(**{args})")
        batch_outputs.append(
            {
                "tool_name": name,
                "output": tool_output
            }
        )
    return batch_outputs

batch_tool_schema = {
    "name": "batch_tool",
    "description": "Invoke multiple other tool calls simultaneously",
    "input_schema": {
        "type": "object",
        "properties": {
            "invocations": {
                "type": "array",
                "description": "The tool calls to invoke",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The name of the tool to invoke",
                        },
                        "arguments": {
                            "type": "string",
                            "description": "The arguments to the tool, encoded as a JSON string",
                        },
                    },
                    "required": ["name", "arguments"],
                },
            }
        },
        "required": ["invocations"],
    },
}


In [91]:
messages = []
tools = [get_current_datetime_schema, add_duration_to_datetime_schema, set_reminder_schema, batch_tool_schema]

add_user_message(messages, "Set a reminder for my doctor's appointment for Jan 1st 2026 15:00. Also, an appointment with a friend the same day, 3 hours later.")

resp = run_conversation(messages, tools)

I'll set both reminders for you - one for your doctor's appointment on January 1st, 2026 at 3:00 PM, and another for your friend appointment 3 hours later at 6:00 PM.

----
Setting the following reminder for 2026-01-01T15:00:00:
Doctor's appointment
----
----
Setting the following reminder for 2026-01-01T18:00:00:
Appointment with friend
----


Perfect! I've successfully set both reminders for you:

1. **Doctor's appointment** - January 1st, 2026 at 3:00 PM (15:00)
2. **Appointment with friend** - January 1st, 2026 at 6:00 PM (18:00) 

Both reminders are now scheduled and you'll receive notifications at the specified times.

In [ ]:
print_messages(resp)

user:  Set a reminder for my doctor's appointment. It's 177 days after Jan1st, 2050. 

assistant:  [TextBlock(citations=None, text="I'll help you set a reminder for your doctor's appointment. First, I need to calculate the date that is 177 days after January 1st, 2050, and then I can set the reminder for you.", type='text'), ToolUseBlock(id='toolu_012xSC3CUUeEty5jVmfbW7By', input={'datetime_str': '2050-01-01', 'duration': 177, 'unit': 'days'}, name='add_duration_to_datetime', type='tool_use')] 

user:  [{'type': 'tool_result', 'tool_use_id': 'toolu_012xSC3CUUeEty5jVmfbW7By', 'content': '"Monday, June 27, 2050 12:00:00 AM"', 'is_error': False}] 

assistant:  [TextBlock(citations=None, text="Now that I have the date for your appointment, I'll set a reminder for you.", type='text'), ToolUseBlock(id='toolu_01SKXXYRKnATrQ2NVzUNx1sL', input={'content': "Doctor's appointment", 'timestamp': '2050-06-27T00:00:00'}, name='set_reminder', type='tool_use')] 

user:  [{'type': 'tool_result', 'tool_u

#### 4.4 Structured Output

**Controlling Tool Use**
Tool choice parameter: force Claude to call a tool.

tool_choice = 
* {"type": "auto"} --> Model decides if it needs to use a tool (default)
* {"type": "any"}  --> Model **must** use a tool - it can decide which to use
* {"type": "tool", "name": "TOOL_NAME"} --> model **must** use the listed tool.

Instead of prompt-based structured outputs, there can be a tool to control the output format.
* It's usually more reliable to use tools, however, it's more complex to implement.
* You need to provide a schema that describes the structure of the desired output.

Ex:
Provide a prompt that specifies which tool to call. The input of the tool can be a structured json, which will force the model to structure data to that specific input format.

In [ ]:
messages = []

add_user_message(
    messages,
    """
    Write a one-paragraph scholarly article about computer science.
    Include a title and author name.
    """
)

response = chat(messages)

In [23]:
Markdown(text_from_message(response))

# Advancements in Quantum Computing: Bridging Theoretical Possibilities and Practical Applications

By Dr. Elena Martinez

Quantum computing represents one of the most promising frontiers in computer science, offering computational capabilities that transcend the limitations of classical computing systems. While traditional computers operate on bits that exist in binary states (0 or 1), quantum computers leverage quantum bits or "qubits" that can exist in multiple states simultaneously through quantum superposition, enabling them to process exponentially more information. Recent breakthroughs in quantum error correction and the development of more stable qubit architectures have significantly reduced decoherence issues that previously hindered practical applications. These advancements are particularly relevant for complex computational problems in cryptography, molecular simulation, and optimization algorithms that remain intractable for classical computers. As researchers continue to refine quantum hardware and develop specialized quantum algorithms, we approach a critical threshold where quantum advantage—the point at which quantum computers outperform their classical counterparts for specific applications—may transition from theoretical possibility to technological reality, potentially revolutionizing fields ranging from materials science to artificial intelligence and secure communications.

In [24]:
# Adding tool choice option
def chat(messages: list,
        model: str = model,
        max_tokens: int = 1000,
        temperature: float = 1.0,
        stop_sequences = [],
        system: str | None = None,
        tools: list | None = None,
        tool_choice = None):
    params = {
        "model": model,
        "max_tokens": max_tokens,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences
    }
    if system:
        params["system"] = system
    if tools:
        params["tools"] = tools
    if tool_choice:
        params["tool_choice"] = tool_choice
    
    response = client.messages.create(**params)
    return response


In [ ]:
# Creating a schema for a function - note: we just want the input (structured json)
article_summary_schema = {
    "name": "article_summary",
    "description": "Creates a summary of an article with its key insights. Use this tool when you need to generate a structured summary of an article, research paper, or any textual content. The tool requires the article's title, author name, and a list of the most important insights or takeaways from the content. Each insight should be a concise statement capturing a significant point from the article.",
    "input_schema": {
        "type": "object",
        "properties": {
            "title": {
                "type": "string",
                "description": "The title of the article being summarized.",
            },
            "author": {
                "type": "string",
                "description": "The name of the author who wrote the article.",
            },
            "key_insights": {
                "type": "array",
                "items": {"type": "string"},
                "description": "A list of the most important takeaways or insights from the article. Each insight should be a complete, concise statement.",
            },
        },
        "required": ["title", "author", "key_insights"],
    },
}

In [28]:
messages = []
add_user_message(messages, text_from_message(response))
response_tool = chat(messages,
    tools = [article_summary_schema],
    tool_choice = {"type":"tool", "name":"article_summary"}
    ) # must use article summary

In [32]:
response_tool.content[0].input

{'title': 'Advancements in Quantum Computing: Bridging Theoretical Possibilities and Practical Applications',
 'author': 'Dr. Elena Martinez',
 'key_insights': ['Quantum computers use qubits that can exist in multiple states simultaneously through quantum superposition, exponentially increasing information processing capacity compared to classical bits.',
  'Recent breakthroughs in quantum error correction and stable qubit architectures have reduced decoherence issues that previously limited practical applications.',
  'Quantum computing shows particular promise for complex computational problems in cryptography, molecular simulation, and optimization algorithms.',
  "The field is approaching a critical threshold where 'quantum advantage' may transition from theoretical possibility to technological reality.",
  'Successful implementation of quantum computing could revolutionize fields including materials science, artificial intelligence, and secure communications.']}

#### 4.5 Fine Grained Tool Calling

There's the option to stream the output, handling **InputJsonEvent**.

<code>
<pre>for chunk in stream:
  if chunk.type == "input_json":
    # Process the partial JSON chunk
    print(chunk.partial_json)
    # Or use the complete snapshot so far
    current_args = chunk.snapshot
</code>

JSON outputs will be delivered in Key-Value pairs, as Claude validates each of them before sending them.

Fine-grained tool calling does one main thing: it disables JSON validation on the API side. This means:

- You get chunks as soon as Claude generates them
- No buffering delays between top-level keys
- More traditional streaming behavior

* Critical: JSON validation is disabled - your code must handle invalid JSON
Enable it by adding fine_grained=True to your API call:

## Text Edit Tool

- Create, read, edit both directories and files.
- View file/directory contents.
- View specific range of lines in a file.
- Replace the text in a file.
- Create new files.
- Insert text at specific lines in a file.
- Undo recent edits to files.

**How to:**
- Claude has the *text_editor_tool_schema*; the function to handle Claude's requests has to be written.

In [7]:
# Implementation of the Text Editor Tool
from lib.TextEditorToolClass import TextEditorTool

# Process Tool Call Requests
text_editor_tool = TextEditorTool()

def run_tool(tool_name, tool_input):
    if tool_name == "str_replace_editor":
        command = tool_input["command"]
        if command == "view":
            return text_editor_tool.view(
                tool_input["path"], tool_input.get("view_range")
            )
        elif command == "str_replace":
            return text_editor_tool.str_replace(
                tool_input["path"], tool_input["old_str"], tool_input["new_str"]
            )
        elif command == "create":
            return text_editor_tool.create(
                tool_input["path"], tool_input["file_text"]
            )
        elif command == "insert":
            return text_editor_tool.insert(
                tool_input["path"],
                tool_input["insert_line"],
                tool_input["new_str"],
            )
        elif command == "undo_edit":
            return text_editor_tool.undo_edit(tool_input["path"])
        else:
            raise Exception(f"Unknown text editor command: {command}")
    else:
        raise Exception(f"Unknown tool name: {tool_name}")

def run_tools(message):
    tool_requests = [
        block for block in message.content if block.type == "tool_use"
    ]
    tool_result_blocks = []

    for tool_request in tool_requests:
        try:
            tool_output = run_tool(tool_request.name, tool_request.input)
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": json.dumps(tool_output),
                "is_error": False,
            }
        except Exception as e:
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": f"Error: {e}",
                "is_error": True,
            }

        tool_result_blocks.append(tool_result_block)

    return tool_result_blocks

In [8]:
# Make the text edit schema based on the model version being used
def get_text_edit_schema(model):
    if model.startswith("claude-3-7-sonnet"):
        return {
            "type": "text_editor_20250124",
            "name": "str_replace_editor",
        }
    elif model.startswith("claude-3-5-sonnet"):
        return {
            "type": "text_editor_20241022",
            "name": "str_replace_editor",
        }
    else:
        raise ValueError(
            f"Editor schema version not known for model: {model}. Reference Anthropic docs for the correct schema version."
        )

In [9]:
# Run the conversation in a loop until the model doesn't ask for a tool use
def run_conversation(messages):
    while True:
        response = chat(
            messages,
            tools=[get_text_edit_schema(model)],
        )

        add_assistant_message(messages, response)
        print(text_from_message(response))

        if response.stop_reason != "tool_use":
            break

        tool_results = run_tools(response)
        add_user_message(messages, tool_results)

    return messages

In [ ]:
messages = []

add_user_message(
    messages,  "Open the ./04_ToolUse_outputs/main_test.py file and summarise its contents.")

resp = run_conversation(messages)

I'll help you view the `main_test.py` file and provide a summary of its contents. Let me open the file first.
### Summary of main_test.py:

This file contains two Python functions:

1. **greetings()**: A simple function that prints "Hey worldie" when called. It doesn't take any parameters or return any values.

2. **fibonacci(n)**: A function that calculates the first n numbers of the Fibonacci sequence.
   - It takes one parameter: `n` (an integer) representing the number of Fibonacci numbers to generate
   - It returns a list containing the first n Fibonacci numbers
   - It includes proper documentation with docstrings explaining the purpose, arguments, and return value
   - It handles edge cases:
     - For n ≤ 0: Returns an empty list
     - For n = 1: Returns [0]
     - For n = 2: Returns [0, 1]
   - For n > 2: It builds the sequence by adding the two previous numbers until it reaches n elements

The file seems to be a test or demonstration file that includes a basic printing func

In [17]:
print_messages(resp[1:])

assistant:  [TextBlock(citations=None, text="I'll help you view the `main_test.py` file and provide a summary of its contents. Let me open the file first.", type='text'), ToolUseBlock(id='toolu_015z7ukpcSLuKo9arxeJKCw5', input={'command': 'view', 'path': './main_test.py'}, name='str_replace_editor', type='tool_use')] 

user:  [{'type': 'tool_result', 'tool_use_id': 'toolu_015z7ukpcSLuKo9arxeJKCw5', 'content': '"1: def greetings():\\n2:     print(\\"Hey worldie\\")\\n3: \\n4: def fibonacci(n):\\n5:     \\"\\"\\"\\n6:     Calculate the first n numbers of the Fibonacci sequence.\\n7:     \\n8:     Args:\\n9:         n (int): The number of Fibonacci numbers to generate\\n10:         \\n11:     Returns:\\n12:         list: A list containing the first n Fibonacci numbers\\n13:     \\"\\"\\"\\n14:     if n <= 0:\\n15:         return []\\n16:     elif n == 1:\\n17:         return [0]\\n18:     elif n == 2:\\n19:         return [0, 1]\\n20:     \\n21:     fib_sequence = [0, 1]\\n22:     for i i

In [ ]:
messages = []

add_user_message(
    messages,  "Open the ./04_ToolUse_outputs/main_test.py file and write a function to calculate the first n numbers of a fibonnacci sequence. Then, create a ./04_ToolUse_outputs/test.py file to test your implementation.")

resp = run_conversation(messages)

I'll help you with that. Let me start by checking if `./main_test.py` exists and then implement a Fibonacci sequence function.
Now I'll modify the `main_test.py` file to add a function that calculates the first n numbers of a Fibonacci sequence:
Now, let me create a test file to test the Fibonacci function implementation:
I've made the following changes:

1. Added a `fibonacci(n)` function to the `main_test.py` file that:
   - Handles edge cases (n ≤ 0, n = 1, n = 2)
   - Returns a list containing the first n numbers of the Fibonacci sequence
   - Uses an iterative approach to build the sequence

2. Created a new `test.py` file that:
   - Imports the fibonacci function from main_test.py
   - Contains test cases using Python's unittest framework
   - Tests various scenarios including:
     - Empty sequence (n ≤ 0)
     - Single element (n = 1)
     - Two elements (n = 2)
     - Multiple elements
     - Larger sequence (n = 10)

You can run the tests by executing `python test.py` in your

## Web Search Tool

In [ ]:
# Web Search Schema
# give some instructions, in the background it will be expanded to a larger schema
web_search_schema = {
    "type": "web_search_20250305",
    "name": "web_search",
    #number of times claude can run a search, depending on the content of the given results
    "max_uses": 5,
    # Constrain Claude's search (avoid AI generated content)
    "allowed_domains": ["nih.gov"]
}

In [19]:
messages = []

add_user_message(messages, "What's the best exercise for loosing belly fat?")

resp = chat(messages, tools = [web_search_schema])

In [36]:
text_response = ''
sources = {}
for content in resp.content:
    if content.type == "text":
        text_response += content.text
        if content.citations:
            for citation in content.citations:
                sources[citation.title] = citation.url

Markdown(text_response)

I'll help you understand exercises for reducing belly fat. Let me search for some evidence-based information on this topic.Based on the search results, I can provide you with evidence-based information about the best exercises for losing belly fat.

# Best Exercises for Losing Belly Fat

## The Truth About Targeted Fat Loss

First, it's important to understand that you can't target fat loss in a specific area. Despite what many fitness products or programs might claim, it's not possible to spot-reduce fat from your belly through exercises that only target that area. If you're hoping to blast away that muffin top around your waist through targeted ab exercises, here's the hard truth — it doesn't work that way. No specific abdominal exercise can eliminate the fat around your midsection.

In order to reduce belly fat, you'll have to reduce the body fat percentage of your whole body. Simply put, to coax your body into utilizing its stored body fat as fuel, you must be burning more calories than you're consuming. Unfortunately, you have no real control over where you'll lose the fat first.

## Most Effective Exercises for Losing Belly Fat

Research points to several types of exercise that are particularly effective:

### 1. Aerobic Exercise (Cardio)

Aerobic exercise (cardio) is an effective way to improve your health and burn calories. Studies suggest that it can be an effective form of exercise for reducing belly fat. The body stores fat as triglycerides, which can be used for fuel anywhere in your body, not just in the exercised area. Doing aerobic exercise while eating a healthy diet is the best way to lose belly fat and overall body fat. This will help to create a calorie deficit (where you use more calories than you consume), which promotes fat loss over time.

Aerobic exercise includes any activity that raises your heart rate such as walking, dancing, running or swimming. This can also include doing housework, gardening and playing with your children.

The frequency and duration of your exercise program are very important. One older 2015 study found that postmenopausal women lost more fat from all areas when they did aerobic exercise for 300 minutes per week, compared with those who exercised 150 minutes per week.

### 2. Strength Training

Adding even moderate strength training to aerobic exercise helps build lean muscle mass, which causes you to burn more calories throughout the entire day, both at rest and during exercise. Aerobics and strength training are two of the best ways to lose belly fat. They are excellent for burning calories and boosting metabolism.

Studies have shown that you can help trim visceral fat or prevent its growth with both aerobic activity (such as brisk walking) and strength training (exercising with weights). Spot exercises, such as sit-ups, can tighten abdominal muscles but won't get at visceral fat.

### 3. High-Intensity Interval Training (HIIT)

The best exercises for losing belly fat mix cardio and strength training. Good options include circuit training and high-intensity interval training (HIIT) for at least 30 minutes a day, 5 days a week.

The burpee is one effective HIIT exercise. This explosive exercise – which entails going from a push-up position to a jump and back to a push-up position – hits every muscle from head to toe. In fact, a study from the American College of Sports Medicine found that 10 fast-paced reps are just as effective at revving your metabolism as a 30-second all

In [34]:
print("Sources: \n")
for title, url in sources.items():
    print(title, ":", url)

Sources: 

Top Exercises for Belly Fat : https://www.webmd.com/fitness-exercise/top-exercises-belly-fat
The #1 Exercise for Belly Fat : https://www.aarp.org/health/healthy-living/how-to-get-rid-of-belly-fat/
27 Exercises to Help Lose Belly Fat | Best Fat Burning Exercises : https://www.menshealth.com/uk/fitness/a758971/the-5-best-exercises-for-burning-belly-fat/
18 Effective Tips to Lose Belly Fat (Backed by Science) : https://www.healthline.com/nutrition/20-tips-to-lose-belly-fat
What's the best exercise to lose fat around your belly? - BHF : https://www.bhf.org.uk/informationsupport/heart-matters-magazine/activity/best-exercise-to-lose-belly-fat
8 Ways to Lose Belly Fat and Live a Healthier Life | Johns Hopkins Medicine : https://www.hopkinsmedicine.org/health/wellness-and-prevention/8-ways-to-lose-belly-fat-and-live-a-healthier-life
Best Exercises to Lose Belly Fat: Aerobics and More - GoodRx : https://www.goodrx.com/conditions/weight-loss/best-exercises-to-lose-belly-fat
Taking Aim

Check out <code>web_search_output.py</code> for full response output.